In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

'Victory Mansions is described as a run-down building with a malfunctioning lift and gritty dust swirling in. The hallway smells of boiled cabbage and old rag mats. The flat is located on the seventh floor, and the building is affected by an electricity cut during daylight hours as part of an economy drive in preparation for Hate Week. Inside the flat, there is a telescreen that cannot be completely shut off, and the building is watched over by a poster with a huge face and the caption "BIG BROTHER IS WATCHING YOU." The protagonist, Winston Smith, lives in this building.'